<a href="https://colab.research.google.com/github/fkatelyn/congressional2023/blob/main/notebooks/palm_tree_gradio_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
MODEL_VERSION = '7'

In [12]:
# prompt: write a gradio which take image input and image output. Use yolov8 from analytics to do the input image processing. The processed image will be the output.

!pip install ultralytics==8.0.134
!pip install gradio






In [13]:
import os
HOME = os.getcwd()
HOME = '/content'
print(HOME)

from google.colab import drive
GOOGLE_DRIVE = f'{HOME}/drive'
drive.mount(GOOGLE_DRIVE)

/content
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
class Experiment:
  '''
  Google drive path:

  PROJECT_DRIVE = MyDrive/ai/projects/palmtree
  EXPERIMENT_DRIVE = MyDrive/ai/projects/palmtree/experiments
  UPLOAD_DRIVE = MyDrive/ai/projects/palmtree/uploads
  DATASET_DRIVE = MyDrive/ai/projects/palmtree/datasets
  DEMO_IMAGES_DRIVE = MyDrive/ai/projects/palmtree/demo/images

  '''
  EPOCHS = 200
  IMGSZ = 640
  CONFIDENT = 0.15
  MODEL_NAME = "yolov8s"
  EXPERIMENT_HOME = f'{HOME}/experiments/{MODEL_VERSION}'
  DATASET_HOME = f'{HOME}/datasets/palm-tree-detection-{MODEL_VERSION}'
  WEIGHTS_HOME = f'{HOME}/weights/{MODEL_VERSION}'

  PROJECT_DRIVE = f'{GOOGLE_DRIVE}/MyDrive/ai/projects/palmtree'
  EXPERIMENT_DRIVE = f'{PROJECT_DRIVE}/experiments/{MODEL_VERSION}'
  DATASET_DRIVE = f'{PROJECT_DRIVE}/datasets/palm-tree-detection-{MODEL_VERSION}'
  WEIGHTS_DRIVE = f'{PROJECT_DRIVE}/weights/{MODEL_VERSION}'
  IMAGES_DEMO_DRIVE = f'{PROJECT_DRIVE}/demo/images'

  PROJECT_SHARED = f'{GOOGLE_DRIVE}/MyDrive/ai/inspiritai/palmtree'
  IMAGES_DEMO_SHARED = f'{PROJECT_SHARED}/demo/images'
  WEIGHTS_SHARED = f'{PROJECT_SHARED}/weights/{MODEL_VERSION}'



print(Experiment.EXPERIMENT_HOME)


/content/experiments/7


In [15]:

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.134 🚀 Python-3.10.12 torch-2.0.1+cu118 CPU
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 27.4/225.8 GB disk)


In [8]:
%cd {HOME}
%mkdir -p {HOME}/weights/
!cp -fpR {Experiment.WEIGHTS_DRIVE} /content/weights/


/content


In [9]:
from PIL import Image
import gradio as gr
import torch
from ultralytics import YOLO
import glob

model = YOLO(f'{Experiment.WEIGHTS_HOME}/best.pt')

def yolo(im, confident, size=Experiment.IMGSZ):
    g = (size / max(im.size))  # gain
    im = im.resize((int(x * g) for x in im.size), Image.LANCZOS)

    results = model.predict(im, conf=confident)
    im_array = results[0].plot()  # updates results.imgs with boxes and labels
    imo = Image.fromarray(im_array[..., ::-1])  # RGB

    return imo



title = "Palm Oil Tree detection"
description = "Upload an image or click an example image to use."
examples = glob.glob(f'{Experiment.DATASET_DRIVE}/test/images/*.jpg')
examples.extend(glob.glob(f'{Experiment.DATASET_DRIVE}/train/images/*.jpg'))
examples.extend(glob.glob(f'{Experiment.DATASET_DRIVE}/valid/images/*.jpg'))
examples = list(map(lambda x: [x], examples))


gr.Interface(fn=yolo,
             inputs=[gr.inputs.Image(type='pil', label="Original Image"),
                     gr.Slider(0.0, 1.0, 0.5)],
             outputs=gr.outputs.Image(type="pil", label="Output Image"),
             title=title,
             description=description,
             examples=examples,
             analytics_enabled=False).launch(debug=True)

<ipython-input-9-41a9ca522d49>:30: GradioDeprecationWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  inputs=[gr.inputs.Image(type='pil', label="Original Image"),
<ipython-input-9-41a9ca522d49>:30: GradioDeprecationWarning: `optional` parameter is deprecated, and it has no effect
  inputs=[gr.inputs.Image(type='pil', label="Original Image"),
<ipython-input-9-41a9ca522d49>:32: GradioDeprecationWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  outputs=gr.outputs.Image(type="pil", label="Output Image"),


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>


0: 640x640 2 bananas, 1 cattle, 2 ganodermas, 10 healthys, 119 nitrogens, 810.0ms
Speed: 25.0ms preprocess, 810.0ms inference, 31.2ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 ganodermas, 1 healthy, 89 nitrogens, 623.8ms
Speed: 4.1ms preprocess, 623.8ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 ganodermas, 40 nitrogens, 1023.0ms
Speed: 3.5ms preprocess, 1023.0ms inference, 1.7ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 ganodermas, 16 nitrogens, 1054.0ms
Speed: 3.2ms preprocess, 1054.0ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 ganoderma, 2 nitrogens, 626.1ms
Speed: 3.9ms preprocess, 626.1ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 nitrogen, 708.5ms
Speed: 6.6ms preprocess, 708.5ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 117 nitrogens, 633.2ms
Speed: 4.4ms preprocess, 633.2ms inference, 1.7ms postp

Keyboard interruption in main thread... closing server.
